In [ ]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from keras.utils import Sequence
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from keras_contrib.layers import CRF
from numpy import newaxis
from random import shuffle
import math
import sklearn
import subprocess
import fastText
import pickle

In [ ]:
def getCasing(word, caseLookup):
    
    if word == 'PADDING_TOKEN':
        return(caseLookup['PADDING_TOKEN'])
    
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
   
    return caseLookup[casing]

In [ ]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [ ]:
def get_sentences_germeval(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence)
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            temp = [splits[1],splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [ ]:
# preproecessing data from Conll
def get_sentences_conll(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [ ]:
trainSentences = get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


In [ ]:
print(testSentences[0])

In [ ]:
labelSet = set()
characters= set()
max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > max_sequence_length:
            max_sequence_length = len(sentence)

In [ ]:
print(len(labelSet))
print(max_sequence_length)

In [ ]:
# :: Create a mapping for the labels ::
label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [ ]:
print(label2Idx)

In [ ]:
# :: Hard coded case lookup ::
case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [ ]:
print(caseEmbeddings)
print(case2Idx)

In [ ]:
print(type(trainSentences))

In [ ]:
print(trainSentences[0])

In [ ]:
char2Idx={"PADDING_TOKEN":0}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

In [ ]:
print(trainSentences[0]) 

In [ ]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [ ]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

In [ ]:
print(trainSentences[0])

In [ ]:
def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_sequences(model, sentences):
    steps = 0
    true_labels = []
    pred_labels = []
    all_true_labels = []
    for s in sentences:
        all_true_labels.append([w[1] for w in s])
    all_pred_labels = model.predict_generator(NerSequence(sentences))
    
    for s_id, s in enumerate(all_true_labels):
        not_padded_true = []
        not_padded_pred = []
        predicted_labels = get_label_from_categorical(all_pred_labels[s_id])
        for t_id, t in enumerate(s):
            if t != 'PADDING_TOKEN': # skip PADDING_TOKEN 
                not_padded_true.append(label2Idx[t])
                not_padded_pred.append(predicted_labels[t_id])
        true_labels.append(not_padded_true)
        pred_labels.append(not_padded_pred)
    
    return(true_labels, pred_labels)

In [ ]:
class NerSequence(Sequence):
    def __init__(self, sentence_data, shuffle_data = False, batch_size=32):
        self.sentence_data = sentence_data
        self.shuffle_data = shuffle_data
        self.batch_size = batch_size
        self.indices = np.arange(len(self.sentence_data))
        if self.shuffle_data:
            np.random.shuffle(self.indices)

    def __len__(self):
        return math.ceil(len(self.sentence_data) / self.batch_size)

    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x, batch_y = self.get_processed_data(inds)
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle_data:
            np.random.shuffle(self.indices)
        
    def get_processed_data(self, inds):
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        
        output_labels = []

        for index in inds: 
            sentence = self.sentence_data[index]

            temp_word= []
            temp_casing = []
            temp_char= []

            temp_output=[]

            # padding
            words_to_pad = max_sequence_length - len(sentence)
            for i in range(words_to_pad):
                sentence.append(['PADDING_TOKEN', 'PADDING_TOKEN'])

            # create data input for words
            for word in sentence:
                word, label = word
                temp_output.append(label2Idx[label])

                casing = getCasing(word, case2Idx)
                temp_casing.append(casing)

                if word == 'PADDING_TOKEN':
                    temp_char2=np.array([char2Idx['PADDING_TOKEN']])
                    temp_char.append(temp_char2)
                    word_vector = [0] * nb_embedding_dims
                    temp_word.append(word_vector)
                else:
                    # char
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN'])
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)

                    # word
                    word_vector = ft.get_word_vector(word.lower())
                    # word_vector = ft.get_word_vector(word)
                    temp_word.append(word_vector)

            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            temp_output = to_categorical(temp_output, len(label2Idx))
            output_labels.append(temp_output)

        return([np.array(word_embeddings), 
                np.array(case_embeddings), 
                np.array(char_embeddings)], 
               np.array(output_labels))

In [ ]:
idx2Label = {v: k for k, v in label2Idx.items()}
print(len(label2Idx))
print(len(idx2Label))

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

In [ ]:
nb_char_embedding_dims = 52
def get_model_lstm():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,nb_char_embedding_dims,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    char_lstm = TimeDistributed(Bidirectional(LSTM(50, name = 'char_lstm')))(embed_char_out)
    output = concatenate([words_input, casing, char_lstm])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5, name = 'token_lstm'))(output)
    output = TimeDistributed(Dense(len(label2Idx), name = 'token_dense'))(output)
    crf = CRF(len(label2Idx), name = 'crf')
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

def get_model_3cnn():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1,
                             name='charcnn_' + str(sz)))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52, name = 'charcnn_maxpool'))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5, name='token_lstm'))(output)
    output = TimeDistributed(Dense(len(label2Idx), name = 'token_dense'))(output)
    crf = CRF(len(label2Idx), name = 'crf')
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def __init__(self, model_file):
        self.model_file = model_file
                 
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_sequences(self.model, devSentences)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > -1 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + self.model_file)
            self.max_f1 = f1
            self.model.save(self.model_file)

In [ ]:
tmp_model_filename = 'tmp_generator_NER_best_transfer.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History(tmp_model_filename)
model = get_model_3cnn()

model.load_weights('wiki_POS_pretrained_LSTM-BiLSTM.h5', by_name= True)

In [ ]:
model.fit_generator(
    NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
    validation_data = NerSequence(devSentences, batch_size=256), 
    epochs = 15, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
tmp_model_filename = 'tmp_generator_NER_best_BIGBATCH_transfer.h5'
model.fit_generator(
    NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
true_labels, pred_labels = predict_sequences(testSentences)
print(compute_f1(pred_labels, true_labels, idx2Label))

# Experiments

In [ ]:
f = open('results_fit_bigbatch_lstm.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_bigbatch_lstm_best.' + str(run_i) + '.h5'

    history = F1History(tmp_model_filename)
    
    # model = get_model_3cnn()
    model = get_model_lstm()
    model.fit_generator(
        NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
        validation_data = NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    true_labels, pred_labels = predict_sequences(testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_generator_NER_bigbatch_best.5.h5', 'final_model_germeval_outer.h5')
with open("final_model_germeval.indexes", "w") as f:
    json.dump([idx2Label, label2Idx, char2Idx, case2Idx], f)

In [ ]:
from keras.models import load_model
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

model = load_model('tmp_generator_NER_bigbatch_best.1.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = predict_sequences(testSentences)
print(compute_f1(pred_labels, true_labels, idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close